In [1]:
import psycopg2
import pandas as pd

In [12]:
db_connection = psycopg2.connect(host="localhost",dbname="airflow",port=5532,user="postgres",password="postgres")

In [6]:
params = {
    "host": "localhost",
    "dbname": "airflow",
    "port": 5532,
    "user": "postgres",
    "password": "postgres"
}
connection = None

try:
    connection = psycopg2.connect(**params)
    print("Connected to the database successfully.")
except Exception as e:
    print(f"An error occurred while connecting to the database: {e}")


Connected to the database successfully.


In [21]:
import psycopg2
import pandas as pd

# Create a function that takes string input of a SQL query and output a dataframe contain the desired data.
def query_from_db(query):
    # Connection to the posrgreSQL in the airflow container.
    connection = psycopg2.connect(
            host="localhost",
            dbname="airflow",
            port=5532,
            user="postgres",
            password="postgres",
        )
    
    cursor = connection.cursor()
    cursor.execute(query)

    # Get the data from the cursor and make a dataframe for it.
    history_rows = cursor.fetchall()
    df = pd.DataFrame(history_rows)

    # Close the connection,
    cursor.close()
    connection.close()

    return df

In [22]:
df = query_from_db("SELECT id, timestamp, price, market_cap, total_volume from coingecko_historic WHERE timestamp >= 1678078892240 AND timestamp < 1683342049799")

In [23]:
df

,0,1,2,3,4
0,None,1678078892240,22362.00823955405,431898530552.6892,15497348287.882515
1,None,1678082525312,22375.039455087837,432073636503.6055,15965619394.74735
2,None,1678086057189,22422.125634818287,433026518971.614,16336096947.327412
3,None,1678089776175,22413.812768997555,432976409654.98444,16541275982.752655
4,None,1678093343893,22431.198532520255,433137313807.4554,16799194209.257723
...,...,...,...,...,...
8773,bitcoin,1683324020324,29530.151331551937,571731122345.3936,18038758113.486103
8774,bitcoin,1683327646632,29642.231838723994,573695108243.3298,7941832613.386951
8775,bitcoin,1683331228148,29520.322268970496,571520224182.8824,4971523630.54862
8776,bitcoin,1683334801988,29724.410733072382,575948431256.4188,12620769574.944633
